# 2-D Diffusionsgleichung (Stationär)

## Einführung

Unter der Annahme, dass sich die Temperatur nicht mehr mit der Zeit verändert, also das Temperaturfeld stationär ist, vereinfacht sich die Wärmeleitgleichung in eine reine Diffusionsgleichung in der Form einer sogenannte Poissongleichung (nur radiogene Elemente):

$$
0 = \frac{\partial}{\partial{x}} \left(k_{x} \frac{\partial{T}}{\partial{x}}\right) + \frac{\partial}{\partial{y}} \left(k_{y} \frac{\partial{T}}{\partial{y}}\right) + \rho H, \tag{1}
$$

wobei $\rho$ die Dichte, $k_{x,y}$ die Wärmeleitfähigkeit in x- und y-Richtung, und $H$ die radiogene Wärmeproduktion pro Masse [W/kg] ist. 

Ein relative bekanntes, vereinfachtes Beispiel, auf das diese Gleichung angewand werden kann, ist das Problem der atomaren Endlagerung. Die Gleichung bestimmt dabei, in erster Ordnung, die finale Temperaturverteilung für ein Problem mit bestimmten Anfangs- und Randbedingungen.

Vereinfachen wir dafür die Gleichung erst noch ein wenig und nehmen an, dass die thermischen Parameter (hier, die Wärmeleitfähigkeit $k$) **isotrop** und **konstant** ist:

$$
0 = \left( \frac{\partial^2{T}}{\partial{x}^2} + \frac{\partial^2{T}}{\partial{y}^2} \right) + \frac{Q}{k}, \tag{2}
$$

wobei $Q = \rho H$ die Wärmeproduktionsrate pro Volumen [W/m^3] ist. 

## Das Problem

... Beschreibung ...

... Loesung ...

In [1]:
using GeoModBox.HeatEquation.TwoD, ExtendableSparse, Plots

Physikalische Parameter

In [ ]:
# Physikalischer Parameter ---------------------------------------------- #
P       = ( 
    L       =   4.0e3,      #   [m]
    H       =   2.0e3,      #   [m]
    k       =   5.6,        #   Waermeleitfaehigkeit, W/m/K
    # Definiere die Region der Waermequelle
    Wcave   =   200.0,      # Breite der Region [ m ]
    Hcave   =   200.0,      # Mächtigkeit [ m ]
    Dcave   =   1.0e3,      # Tiefe des Zentrums [ m ]
    Xcave   =   2.0e3,      # x-Position des Zentrum [ m ]
    Q       =   0.3         # Volumetrische Waermeproduktionsrate [ W/m³ ]; Q = rho*H
)
# ----------------------------------------------------------------------- #

Numerische Parameter

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   640,        # Gitterpunkte in x-Richtung, Spalten
    y       =   320         # Gitterpunkte in y-Richtung, Zeilen    
    
)

Δ       = (
    x       =   [0.0],      #   Gitterabstand in x-Richtung
    y       =   [0.0]       #   Gitterabstand in y-Richtung
)

Δ.x[1]      =   P.L/NC.x
Δ.y[1]      =   P.H/NC.y
# ----------------------------------------------------------------------- #

Modellparameter und Anfangsbedingungen

In [4]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(0.0 + Δ.x[1]/2.0, P.L - Δ.x[1]/2.0, NC.x),
)
y       = (
    c       =   LinRange(-P.H + Δ.y[1]/2.0, 0.0 - Δ.y[1]/2.0, NC.y),
)
# ----------------------------------------------------------------------- #
# Erstellung des Anfangsbedingung --------------------------------------- #
D       = ( 
    Q       =   zeros(NC...),           # (row,col) 
    T       =   zeros(NC...),
    ind     =   zeros(Int64,NC...)
)

# Defniere die Region of additional heat source
for i = 1:NC.x, j = 1:NC.y
    if x.c[i] >= (P.Xcave-P.Wcave/2.0) && x.c[i] <=(P.Xcave+P.Wcave/2.0) && 
        y.c[j] >= -P.Dcave-P.Hcave/2.0 && y.c[j] <= -P.Dcave+P.Hcave/2.0 
        D.ind[i,j]  =   1
        D.Q[i,j]    = P.Q
    else
        D.ind[i,j]  =   0
    end
end
# ----------------------------------------------------------------------- #

## Die Lösung 

### Diskretisierung

#### Gitter und Indizierung

#### Finite Differenzen Approximation

$$
0 = \left( \frac{T_{i-1,j} - 2T_{i,j} + T_{i+1,j}}{\Delta{x}^2} + \frac{T_{i,j-1} - 2T_{i,j} + T_{i,j+1}}{\Delta{y}^2} \right) + \frac{Q}{k}, \tag{3}
$$

wobei $i, j$ jeweils die Indizes und $\Delta{x}, \Delta{y}$ die Gitterabstände in x- und y- Richtung sind. Durch Umformung erhalten wir ein lineares Gleichungsystem mit 5 Koeffiziente in der Form: 

$$
b T_{i,j-1} + aT_{i-1,j} - (2a + 2b) T_{i,j} + a T_{i+1,j} + b T_{i,j+1} = - \frac{Q}{k}, \tag{4}
$$

wobei $ a = 1 / \Delta{x}^2$ und $b = 1 / \Delta{y}^2$ ist. 

### Randbedingungen

... Änderung der Koeffizienten und der Rechten Seite

**Dirichlet** 
*West*
*East*
*South*
*North*

**Neumann**
*West*
*East*
*South*
*North*


In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (
    type    = (W=:Dirichlet, E=:Dirichlet, N=:Dirichlet, S=:Dirichlet),
    # type    = (W=:Neumann, E=:Neumann, N=:Neumann, S=:Neumann),
    # type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
    # type    = (N=:Neumann, S=:Neumann, W=:Dirichlet, E=:Dirichlet),
    val     = (W=:0.0,E=:0.0,N=:0.0,S=:0.0)
)
# ----------------------------------------------------------------------- #

Lineares Gleichungsystem ...

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(Num.T)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# ----------------------------------------------------------------------- #

... programmiert in ```Poisson!()``` ...

### Loesen der Problems ...

In [ ]:
# Solve equation -------------------------------------------------------- #
Poisson!(D,NC,P,BC,Δ,K,rhs,Num)
# ----------------------------------------------------------------------- #

## Visualisierung

In [ ]:
# Plot solution --------------------------------------------------------- #
p = heatmap(x.c ./ 1e3, y.c ./ 1e3, D.T', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Stationary temperature field", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 900))

contour!(p, x.c ./ 1e3, y.c ./ 1e3, D.T', 
            levels=100:100:1500, linecolor=:black)

display(p)

savefig("./Results/04_Steady_State_Solution.png")
# ----------------------------------------------------------------------- #